In [278]:
# 이 전처리는 개체 기준은 한글명 + 정당을 기준으로 합니다. 같은 정당, 같은 한글명이 있는 경우 추가적인 전처리가 필요합니다.

In [279]:
import pandas as pd
import pickle
import numpy as np
from tqdm import tqdm

In [281]:
from datetime import datetime,timedelta

In [280]:
for start in pd.date_range('2020-06-01','2023-09-26'):
    ## 사용할 기간을 지정합니다.
    end = start + timedelta(90)

    ## 데이터 불러오기
    with open('data/raw2.pickle', 'rb') as f:
        raw2 = pickle.load(f)

    ## 데이터 복사
    raw3 = raw2.copy()

    ## 사용할 기간에 따라 데이터를 추출합니다

    raw3 = raw3[raw3['date'].between(start, end)]

    ## 데이터 구분
    ##[이름,당명,한자명] 형식인 데이터를 각각 하나의 열로 구분합니다. 한자명은 사용하지 않습니다.

    raw3['main_name'] = [x[0] for x in raw3['main']]
    raw3['main_party'] = [x[1] for x in raw3['main']]

    raw3 = raw3.explode('members')

    raw3['member_name'] = [x[0] for x in raw3['members']]
    raw3['member_party'] = [x[1] for x in raw3['members']]

    raw3 = raw3.drop(['main','members'],axis = 1)

    ## 당명 변경
    raw3['main_party'] = [x.replace('미래통합당','국민의힘') for x in raw3['main_party']]
    raw3['member_party'] = [x.replace('미래통합당','국민의힘') for x in raw3['member_party']]

    ## 공백 제거
    raw3['main_name'] = [x.replace('\n','') for x in raw3['main_name']]
    raw3['main_name'] = [x.replace('\t','') for x in raw3['main_name']]

    raw3['main_party'] = [x.replace('\n','') for x in raw3['main_party']]
    raw3['main_party'] = [x.replace('\t','') for x in raw3['main_party']]

    raw3['member_name'] = [x.replace('\n','') for x in raw3['member_name']]
    raw3['member_name'] = [x.replace('\t','') for x in raw3['member_name']]

    raw3['member_party'] = [x.replace('\n','') for x in raw3['member_party']]
    raw3['member_party'] = [x.replace('\t','') for x in raw3['member_party']]

    ## 매트릭스 인덱스 이름 설정 기본 : 한글명 + 당명
    raw3['main'] = raw3['main_name'] + ' ' + raw3['main_party']
    raw3['member'] = raw3['member_name'] + ' ' + raw3['member_party']

    matrix = raw3.pivot_table(index="main", columns="member", aggfunc="size", fill_value=0)

    # 전처리된 자료 저장.    
    with open('data/matrix_{}_{}.pickle'.format(str(start)[:10],str(end)[:10]), 'wb') as f:
        pickle.dump(matrix , f, pickle.HIGHEST_PROTOCOL)
